In [2]:
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [3]:
model_path = './models/face_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

In [116]:
from moviepy.editor import *

# Extract video clip
CORPUS_PATH = '/home/lain/Merkel-Podcast-Corpus'
TIMINGS_PATH = os.path.join(CORPUS_PATH, 'timings.txt')
timings = open(TIMINGS_PATH, 'r').readlines()
clip_data = timings[800].split("|")
clip_date, clip_start, clip_end, clip_text, _ = clip_data
clip_start, clip_end = float(clip_start), float(clip_end)
clip = VideoFileClip(os.path.join(CORPUS_PATH, 'corpus', clip_date, 'video.mp4'))
audio_path = os.path.join(CORPUS_PATH, 'corpus', clip_date, 'audio.wav')
clip = clip.subclip(clip_start, clip_end)
clip = clip.subclip(0.0, 3.0)
clip.fps = 25
clip.ipython_display(width=360)

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [106]:
frame = clip.get_frame(0)
frame = np.array(frame)
frame.shape

(480, 640, 3)

In [107]:
with FaceLandmarker.create_from_options(options) as landmarker:
    # Load the input image from a numpy array.
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    # Perform face landmarking on the provided single image.
    # The face landmarker must be created with the image mode.
    face_landmarker_result = landmarker.detect(mp_image)

    print(face_landmarker_result)


FaceLandmarkerResult(face_landmarks=[[NormalizedLandmark(x=0.5588638782501221, y=0.33536094427108765, z=-0.02198486030101776, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5569754242897034, y=0.29780885577201843, z=-0.031184516847133636, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5569074749946594, y=0.3109856843948364, z=-0.01861666329205036, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5499599575996399, y=0.2746138572692871, z=-0.020417718216776848, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5564560294151306, y=0.28936973214149475, z=-0.03206857293844223, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5554946660995483, y=0.2799663841724396, z=-0.028304263949394226, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5529420375823975, y=0.2573778033256531, z=-0.008436321280896664, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.510439395904541, y=0.26480549573898315, z=0.008871289901435375, visibility=0.0, presence=0.0), NormalizedL

I0000 00:00:1710439408.660214 1779876 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710439408.686048 1787737 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce GTX 1070 Ti/PCIe/SSE2
W0000 00:00:1710439408.686408 1779876 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.


In [108]:
face_landmarker_result

FaceLandmarkerResult(face_landmarks=[[NormalizedLandmark(x=0.5588638782501221, y=0.33536094427108765, z=-0.02198486030101776, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5569754242897034, y=0.29780885577201843, z=-0.031184516847133636, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5569074749946594, y=0.3109856843948364, z=-0.01861666329205036, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5499599575996399, y=0.2746138572692871, z=-0.020417718216776848, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5564560294151306, y=0.28936973214149475, z=-0.03206857293844223, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5554946660995483, y=0.2799663841724396, z=-0.028304263949394226, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5529420375823975, y=0.2573778033256531, z=-0.008436321280896664, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.510439395904541, y=0.26480549573898315, z=0.008871289901435375, visibility=0.0, presence=0.0), NormalizedL

In [109]:
from mediapipe.framework.formats import landmark_pb2
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_LIPS
from mediapipe import solutions
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def draw_landmarks(img, result):
  pil_img = Image.fromarray(img)
  draw = ImageDraw.Draw(pil_img)

  mx = np.mean([result.face_landmarks[0][vertex[0]].x for vertex in FACEMESH_LIPS])
  my = np.mean([result.face_landmarks[0][vertex[0]].y for vertex in FACEMESH_LIPS])

  for face in result.face_landmarks:
    for vertex in FACEMESH_LIPS:
      x1, y1 = face[vertex[0]].x, face[vertex[0]].y,
      x2, y2 = face[vertex[1]].x, face[vertex[1]].y,
      draw.line((x1*pil_img.width, y1*pil_img.height, x2*pil_img.width, y2*pil_img.height), fill=(255, 0, 0), width=2)

  draw.point((mx*pil_img.width, my*pil_img.height), fill=(0, 0, 255))
  return pil_img

# # STEP 5: Process the detection result. In this case, visualize it.
# #annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), face_landmarker_result)
# annotated_image = draw_landmarks(mp_image.numpy_view(), face_landmarker_result)
# #plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
# plt.imshow(annotated_image)
# plt.show()

In [110]:
annotated = []
landmarks = []
with FaceLandmarker.create_from_options(options) as landmarker:
    for i in range(50):
        t = i / clip.fps
        frame = clip.get_frame(t)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        result = landmarker.detect(mp_image)

        if len(result.face_landmarks) != 0:
            landmarks.append(result.face_landmarks[0])
            annotated.append(draw_landmarks(mp_image.numpy_view(), result))

I0000 00:00:1710439409.920263 1779876 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1710439409.926670 1787746 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.23.08), renderer: NVIDIA GeForce GTX 1070 Ti/PCIe/SSE2
W0000 00:00:1710439409.927034 1779876 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.


In [111]:
video = concatenate([ImageClip(np.array(plot_frame)).set_duration(1/clip.fps) for plot_frame in annotated])
video.ipython_display(width=360, fps=25)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [112]:
# smoothe mean of the mouth position
mxx = []
myy = []
for i in range(50):
    all_points = []

    for j in FACEMESH_LIPS:
        all_points.append(landmarks[i][j[0]])
        all_points.append(landmarks[i][j[1]])
    mx = np.mean([point.x for point in all_points])
    my = np.mean([point.y for point in all_points])
    mxx.append(mx)
    myy.append(my)

def smooth_positions(positions, window_size=5):
    smoothed = []
    for i in range(len(positions)):
        # Determine the start and end of the window for averaging
        start = max(0, i - window_size + 1)
        end = i + 1
        # Calculate the average within this window
        window_average = np.mean(positions[start:end])
        smoothed.append(window_average)
    return smoothed

# Assuming mxx and myy are filled with the mean x and y positions of the mouth
window_size = 5  # Adjust this value as needed for your smoothing
smoothed_mxx = smooth_positions(mxx, window_size=window_size)
smoothed_myy = smooth_positions(myy, window_size=window_size)

In [113]:
cropped_frames = []
reference_distance = 100

def calculate_angle(point1, point2):
    # Calculate the angle in radians between two points
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    angle_rad = np.arctan2(delta_y, delta_x)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

def rotate_image(image, angle, center=None, scale=1.0):
    # Grab the dimensions of the image and then determine the center
    (h, w) = image.shape[:2]

    if center is None:
        center = (w // 2, h // 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated


for i in range(50):
    t = i / clip.fps
    frame = np.array(clip.get_frame(t))

    frame = cv2.resize(frame, (1280, 720))

    all_points = []

    for j in FACEMESH_LIPS:
        all_points.append(landmarks[i][j[0]])
        all_points.append(landmarks[i][j[1]])

    #mx = np.mean([point.x for point in all_points])
    #my = np.mean([point.y for point in all_points])

    mx = smoothed_mxx[i]
    my = smoothed_myy[i]

    left_corner_index = 207
    right_corner_index = 427

    lip_corner_left = landmarks[i][left_corner_index]  # You need to define left_corner_index
    lip_corner_right = landmarks[i][right_corner_index]  # You need to define right_corner_index
    actual_distance = np.sqrt((lip_corner_right.x - lip_corner_left.x)**2 + (lip_corner_right.y - lip_corner_left.y)**2)
    actual_distance *= frame.shape[1]  # Adjust for frame size

    # angle = calculate_angle(lip_corner_left, lip_corner_right)
    # center_of_rotation = ((lip_corner_left[0] + lip_corner_right[0]) / 2 * frame.shape[1], (lip_corner_left[1] + lip_corner_right[1]) / 2 * frame.shape[0])
    # rotated_frame = rotate_image(frame, -angle, center=center_of_rotation)

    scale_factor = actual_distance / reference_distance

    x = int(mx * frame.shape[1])
    y = int(my * frame.shape[0])
    w = int(96//2*scale_factor)
    h = int(96//2*scale_factor)

    cropped_frame = cv2.resize(frame[y-h:y+h, x-w:x+w], (96,96))

    cropped_frames.append(cropped_frame)

In [114]:
video = concatenate([ImageClip(np.array(plot_frame)).set_duration(1/clip.fps) for plot_frame in cropped_frames])
video.ipython_display(width=360, fps=25)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [13]:
# get mean size of mouth
all_points = set()

for vert in FACEMESH_LIPS:
    all_points.add(vert[0])
    all_points.add(vert[1])
all_points = list(all_points)

In [37]:
# get the max width and height of the mouth
max_dist_x = 0
max_dist_y = 0

for pa in all_points:
    for pb in all_points:
        dist_x = abs(landmarks[0][pa].x - landmarks[0][pb].x)
        if dist_x > max_dist_x:
            max_dist_x = dist_x
        dist_y = abs(landmarks[0][pa].y - landmarks[0][pb].y)
        if dist_y > max_dist_y:
            max_distyx = dist_y

In [38]:
max_dist_x

0.04182779788970947